# Paddle OCR을 실행해보자

In [7]:
# CUDA-11.8이 설치되어 있는 GPU 사용 가능한 컴퓨터기 때문에 gpu 버전 다운로드
!pip install paddlepaddle-gpu

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)


In [8]:
# PaddleOCR whl package 설치
!pip install "paddleocr>=2.0.1"

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)


# 한번 사용해보도록 하자

실행하기 전, 컴퓨터에 cuDNN이 설치되어있는지 확인하자. 없다면 설치

CUDA 11.8 사용 기준으로

wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-keyring_1.1-1_all.deb
sudo dpkg -i cuda-keyring_1.1-1_all.deb
sudo apt-get update
sudo apt-get -y install cudnn
sudo apt-get -y install cudnn-cuda-11

In [9]:
import paddle
print(f"Is compiled with CUDA? {paddle.is_compiled_with_cuda()}")
print(f"CUDA devices:, {paddle.device.cuda.device_count()}")

Is compiled with CUDA? True
CUDA devices:, 1


In [10]:
# 콘솔에 아래 명령 입력
# paddleocr --image_dir .책표지_기타_000001.jpg --lang=korean

In [11]:
from paddleocr import PaddleOCR,draw_ocr

ocr = PaddleOCR(lang="korean")

[2025/03/19 11:10:07] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=True, use_xpu=False, use_npu=False, use_mlu=False, use_gcu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/home/uijong/.paddleocr/whl/det/ml/Multilingual_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/home/uijong/.paddleocr/whl/rec/korean/korean_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 32

In [24]:
# 코드로 실행하자면?
# Paddleocr supports Chinese, English, French, German, Korean and Japanese.
# You can set the parameter `lang` as `ch`, `en`, `fr`, `german`, `korean`, `japan`
# to switch the language model in order.
# ocr = PaddleOCR(use_angle_cls=True, lang='korean') # need to run only once to download and load model into memory
img_path = '22341.jpg'
result = ocr.ocr(img_path, cls=True)
for idx in range(len(result)):
    res = result[idx]
    for line in res:
        print(line)


# draw result
from PIL import Image
result = result[0]
image = Image.open(img_path).convert('RGB')
boxes = [line[0] for line in result]
txts = [line[1][0] for line in result]
scores = [line[1][1] for line in result]
im_show = draw_ocr(image, boxes, txts, scores, font_path='MaruBuri-Regular.ttf')
im_show = Image.fromarray(im_show)
im_show.save('result.jpg')
im_show.show()

[2025/03/19 11:12:21] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
[2025/03/19 11:12:21] ppocr DEBUG: dt_boxes num : 21, elapsed : 0.04074358940124512
[2025/03/19 11:12:21] ppocr DEBUG: rec_res num  : 21, elapsed : 0.08551335334777832
[[[84.0, 4.0], [152.0, 9.0], [150.0, 44.0], [82.0, 40.0]], ('나의', 0.9990434646606445)]
[[[159.0, 1.0], [216.0, 5.0], [213.0, 55.0], [156.0, 51.0]], ('봉을', 0.9286381006240845)]
[[[230.0, 15.0], [357.0, 25.0], [354.0, 64.0], [227.0, 54.0]], ('기억할슥', 0.8464200496673584)]
[[[373.0, 25.0], [471.0, 30.0], [469.0, 71.0], [371.0, 65.0]], ('있을까', 0.9830038547515869)]
[[[89.0, 63.0], [181.0, 70.0], [177.0, 120.0], [86.0, 113.0]], ('봉꽂이', 0.8216092586517334)]
[[[201.0, 80.0], [261.0, 80.0], [261.0, 120.0], [201.0, 120.0]], ('지는', 0.9998247623443604)]
[[[282.0, 85.0], [385.0, 90.0], [383.0, 127.0], [281.0, 122.0]], ('계정어', 0.6298844218254089)]
[[[74.0, 125.0], [171.0, 135.0], [167.0, 176.0], [69.0, 165.0

In [25]:
# for idx in range(len(boxes)):
#     print(f"boxes: {boxes[idx]}, txts: {txts[idx]}")

# 한글 또는 영어, 숫자 txt만 포함된 box로 재구성
boxes = [boxes[idx] for idx in range(len(boxes)) if txts[idx].isalnum()]

for idx in range(len(boxes)):
    print(f"boxes: {boxes[idx]}")

boxes: [[84.0, 4.0], [152.0, 9.0], [150.0, 44.0], [82.0, 40.0]]
boxes: [[159.0, 1.0], [216.0, 5.0], [213.0, 55.0], [156.0, 51.0]]
boxes: [[230.0, 15.0], [357.0, 25.0], [354.0, 64.0], [227.0, 54.0]]
boxes: [[373.0, 25.0], [471.0, 30.0], [469.0, 71.0], [371.0, 65.0]]
boxes: [[89.0, 63.0], [181.0, 70.0], [177.0, 120.0], [86.0, 113.0]]
boxes: [[201.0, 80.0], [261.0, 80.0], [261.0, 120.0], [201.0, 120.0]]
boxes: [[282.0, 85.0], [385.0, 90.0], [383.0, 127.0], [281.0, 122.0]]
boxes: [[74.0, 125.0], [171.0, 135.0], [167.0, 176.0], [69.0, 165.0]]
boxes: [[201.0, 141.0], [304.0, 145.0], [303.0, 181.0], [199.0, 178.0]]
boxes: [[324.0, 143.0], [411.0, 148.0], [409.0, 190.0], [321.0, 184.0]]
boxes: [[74.0, 183.0], [134.0, 183.0], [134.0, 245.0], [74.0, 245.0]]
boxes: [[142.0, 199.0], [205.0, 202.0], [204.0, 238.0], [140.0, 235.0]]
boxes: [[217.0, 202.0], [392.0, 202.0], [392.0, 252.0], [217.0, 252.0]]
boxes: [[405.0, 212.0], [488.0, 216.0], [485.0, 259.0], [403.0, 255.0]]
boxes: [[74.0, 257.0], [19

In [14]:
!pip install --upgrade pip

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)


In [15]:
!pip install opencv-python

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)


In [26]:
import cv2
import numpy as np

# 1. cv2 imread로 다시 그레이스케일 이미지 획득
image = cv2.imread('22341.jpg', cv2.IMREAD_GRAYSCALE)

# 2. 연산 효율성을 올리기 위해 boxes를 numpy array로 변환
boxes_np_arr = np.array([])
if boxes and len(boxes) > 0:
    boxes_np_arr = np.array([np.array(box) for box in boxes])

    for box in boxes:
        print(box)
else:
    print("No boxes")

[[84.0, 4.0], [152.0, 9.0], [150.0, 44.0], [82.0, 40.0]]
[[159.0, 1.0], [216.0, 5.0], [213.0, 55.0], [156.0, 51.0]]
[[230.0, 15.0], [357.0, 25.0], [354.0, 64.0], [227.0, 54.0]]
[[373.0, 25.0], [471.0, 30.0], [469.0, 71.0], [371.0, 65.0]]
[[89.0, 63.0], [181.0, 70.0], [177.0, 120.0], [86.0, 113.0]]
[[201.0, 80.0], [261.0, 80.0], [261.0, 120.0], [201.0, 120.0]]
[[282.0, 85.0], [385.0, 90.0], [383.0, 127.0], [281.0, 122.0]]
[[74.0, 125.0], [171.0, 135.0], [167.0, 176.0], [69.0, 165.0]]
[[201.0, 141.0], [304.0, 145.0], [303.0, 181.0], [199.0, 178.0]]
[[324.0, 143.0], [411.0, 148.0], [409.0, 190.0], [321.0, 184.0]]
[[74.0, 183.0], [134.0, 183.0], [134.0, 245.0], [74.0, 245.0]]
[[142.0, 199.0], [205.0, 202.0], [204.0, 238.0], [140.0, 235.0]]
[[217.0, 202.0], [392.0, 202.0], [392.0, 252.0], [217.0, 252.0]]
[[405.0, 212.0], [488.0, 216.0], [485.0, 259.0], [403.0, 255.0]]
[[74.0, 257.0], [190.0, 262.0], [188.0, 302.0], [72.0, 297.0]]
[[213.0, 264.0], [355.0, 268.0], [354.0, 306.0], [212.0, 303.

In [28]:
# 3. 각 box = bounding box
# bounding box 영역 안에 있는 각 영역을 Crop해서 새로운 이미지로 추출한다
croped_images = []
for idx, box in enumerate(boxes_np_arr):
    # box 점 정렬
    rect = np.zeros((4, 2), dtype="float32")
    s = box.sum(axis=1) # 네 좌표의 합
    diff = np.diff(box, axis=1) # 네 좌표의 차이
    rect[0] = box[np.argmin(s)] # 좌상단 좌표
    rect[2] = box[np.argmax(s)] # 우하단 좌표
    rect[1] = box[np.argmin(diff)] # 우상단 좌표
    rect[3] = box[np.argmax(diff)] # 좌하단 좌표

    # rect = [ 좌상, 우상, 우하, 좌하 ]

    # 네 꼭짓점 좌표를 numpy array로 변환
    # 이때, 검출 영역이 너무 Fit하게 되어있어서 +10을 해준다
    offset = 10
    image_height, image_width = image.shape[:2]
    image_height -= 1
    image_width -= 1

    # rect[0] = 좌상단
    x1 = max(0, rect[0][0] - offset)
    y1 = max(0, rect[0][1] - offset)
    # rect[1] = 우상단
    x2 = min(rect[1][0] + offset, image_width)
    y2 = max(0, rect[1][1] - offset)
    # rect[2] = 우하단
    x3 = min(rect[2][0] + offset, image_width)
    y3 = min(rect[2][1] + offset, image_height)
    # rect[3] = 좌하단
    x4 = max(0, rect[3][0] - offset)
    y4 = min(rect[3][1] + offset, image_height)
    pts = np.array(
        [
            [x1, y1],
            [x2, y2],
            [x3, y3],
            [x4, y4]
        ],
        dtype="float32"
    )

    # 네 좌표를 바탕으로 이미지의 너비 및 높이 계산
    widthA = np.sqrt((pts[2][0] - pts[3][0])**2 + (pts[2][1] - pts[3][1])**2)
    widthB = np.sqrt((pts[1][0] - pts[0][0])**2 + (pts[1][1] - pts[0][1])**2)
    maxWidth = max(int(widthA), int(widthB))

    heightA = np.sqrt((pts[1][0] - pts[2][0])**2 + (pts[1][1] - pts[2][1])**2)
    heightB = np.sqrt((pts[0][0] - pts[3][0])**2 + (pts[0][1] - pts[3][1])**2)
    maxHeight = max(int(heightA), int(heightB))

    # print(maxWidth, maxHeight)

    # 출력될 이미지 좌표 설정
    dst = np.array(
        [
            [0, 0],
            [maxWidth - 1, 0],
            [maxWidth - 1, maxHeight - 1],
            [0, maxHeight - 1]
        ],
        dtype="float32"
    )

    # Perspective Transformation
    M = cv2.getPerspectiveTransform(pts, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

    # 음영 제거 -> 모폴로지 연산
    kernel_size = 150
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))

    background = cv2.morphologyEx(warped, cv2.MORPH_OPEN, kernel)
    gray_minus_bg = cv2.subtract(warped, background)
    normed = cv2.normalize(gray_minus_bg, None, 0, 255, cv2.NORM_MINMAX)

    croped_images.append(normed)
    cv2.imwrite(f"./cropped/croped_{idx}.png", normed)

In [29]:
print(len(croped_images))

21


In [31]:
# 각 이미지 적응형 이진화 수행
# 근데 이미지에 들어있는 글자가 검정색일수도 있고 흰색일수도 있어서
# Crop된 이미지의 평균 픽셀값을 계산해서 평균값이 128보다 작으면 cv2.THRESH_BINARY_INV로 수행

for idx, croped_image in enumerate(croped_images):

    # Median 블러링
    blurred = cv2.medianBlur(croped_image, 3)

    # 적응형 이진화
    # binarization = cv2.adaptiveThreshold(
    #     blurred,
    #     maxValue=255,
    #     adaptiveMethod=cv2.ADAPTIVE_THRESH_MEAN_C,
    #     thresholdType=cv2.THRESH_BINARY,  # 또는 THRESH_BINARY_INV
    #     blockSize=21,   # 주변 픽셀 블록 크기
    #     C=13     # 결과에서 뺄 상수
    # )
    _, binarization = cv2.threshold(blurred, 127, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # 만약 배경이 반전되어 있다면 invert
    if np.mean(binarization) < 128:
        binarization = cv2.bitwise_not(binarization)

    # kernel = np.ones((5, 5), np.uint8)
    # opened = cv2.morphologyEx(binarization, cv2.MORPH_CLOSE, kernel)

    cv2.imwrite(f"./character/character_{idx}.png", binarization)


In [32]:
import os

# Load all images from the character directory
binarizationed_images = []
character_dir = "./character/"
for filename in sorted(os.listdir(character_dir)):
    if filename.endswith((".png", ".jpg", ".jpeg")):  # Correct way to check multiple extensions
        img_path = os.path.join(character_dir, filename)
        # Load as grayscale directly
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        _, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        thresh_inv = cv2.bitwise_not(thresh)

        if thresh_inv is not None:
            binarizationed_images.append(thresh_inv)

print(f"Loaded {len(binarizationed_images)} images")

for idx, image in enumerate(binarizationed_images):
    contours, hierarchy = cv2.findContours(
        image,
        cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE
    )

    # 외곽선 기반으로 각 글자 이미지 추출
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)

        print(f"Contour: {x}, {y}, {w}, {h}")

        # 글자 이미지 추출 및 검정색으로 다시 변환
        character = image[y:y+h, x:x+w]
        character = cv2.bitwise_not(character)
        cv2.imwrite(f"./final/character_{idx}_{x}_{y}_{w}_{h}.png", character)


Loaded 21 images
Contour: 51, 30, 16, 5
Contour: 69, 17, 6, 24
Contour: 39, 13, 23, 24
Contour: 14, 11, 21, 24
Contour: 22, 35, 21, 15
Contour: 49, 33, 14, 18
Contour: 42, 26, 19, 6
Contour: 41, 12, 16, 14
Contour: 16, 9, 25, 26
Contour: 22, 49, 18, 14
Contour: 46, 45, 17, 19
Contour: 18, 37, 41, 9
Contour: 79, 34, 1, 2
Contour: 18, 17, 37, 24
Contour: 62, 18, 10, 24
Contour: 13, 16, 21, 21
Contour: 36, 14, 25, 26
Contour: 164, 38, 11, 16
Contour: 145, 38, 13, 14
Contour: 163, 32, 14, 4
Contour: 132, 31, 12, 14
Contour: 130, 25, 13, 3
Contour: 88, 24, 25, 22
Contour: 47, 24, 16, 15
Contour: 70, 23, 16, 19
Contour: 65, 23, 7, 21
Contour: 113, 22, 12, 24
Contour: 152, 21, 9, 15
Contour: 31, 21, 13, 26
Contour: 15, 20, 21, 18
Contour: 131, 18, 13, 3
Contour: 165, 17, 13, 11
Contour: 37, 27, 13, 14
Contour: 74, 26, 10, 4
Contour: 83, 21, 7, 20
Contour: 52, 21, 23, 20
Contour: 14, 14, 27, 35
Contour: 118, 44, 1, 1
Contour: 14, 36, 24, 13
Contour: 38, 33, 17, 15
Contour: 14, 28, 18, 14
Conto